# Actividad DBSCAN y hierarchical clustering (análisis)

En esta actividad se debe aplicar dbscan y hierarchical clustering para unos datos de videojuegos e interpretar sus resultados. Como ejemplo, se presenta los resultados para el modelo de k-means.

In [ ]:
#Instalando las librerias requeridas
#import sys
#!{sys.executable} -m pip install numpy, pandas, plotnine, sklearn

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

Los datos corresponden a distintas ventas de videojuegos en el mundo, así como las evaluaciones de la crítica y usuarios y otros features. Para esta actividad, nos limitaremos a usar los datos númericos, que corresponden a las ventas y las críticas.

También, escalaremos los datos entre 0 y 1, usando la función `MinMaxScaler` de sklearn.

In [ ]:
#Cargando set de datos y haciendo limpieza
origData=pd.read_csv("video_games_sales.csv")
origData=origData.dropna() #Eliminando datos con valores NA 

#eliminando las variables discretas y transformando el resto a númerico
origDataNumeric=origData.iloc[:,[5,6,7,8,10,12]].copy()

#Estandarizando las variables entre 0 y 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
newData = pd.DataFrame(scaler.fit_transform(origDataNumeric), index = origDataNumeric.index, columns=origDataNumeric.columns)

In [ ]:
newData

Apliquemos K-Means para ver el resultado que nos entrega.

In [ ]:
#Aplicando K-means
from sklearn.cluster import KMeans
sse = []
maxK=20
for k in range(1, maxK):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(newData)
    sse.append(kmeans.inertia_)
pd.DataFrame(sse, index=np.arange(1,maxK)).plot()
# tempDF = pd.DataFrame(range(1, maxK),columns=["numK"])
# tempDF["sse"]=sse
# (ggplot(tempDF)+aes(x="numK",y="sse")+theme_bw()+geom_line()+labs(x="Número de clusters",y="WCD")
#  +scale_x_continuous(breaks=range(1,maxK)))

In [ ]:
#Seleccionando entre 5 y 10 clusters
numK=5
finalModel=KMeans(n_clusters=numK)
finalModel=finalModel.fit(newData)


Veamos las estadísticas de cada cluster (media y varianza)

In [ ]:
newData.groupby(finalModel.labels_).agg(['mean','std'])

Finalmente, intentemos visualizar los clusters usando PCA

In [ ]:
#Visualizando los clusters con PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca = pca.fit(newData)
tempData = pca.transform(newData)
tempData = pd.DataFrame(tempData,columns=["PC1","PC2"])
tempData["labels"]=finalModel.labels_
ggplot(tempData)+aes(x="PC1",y="PC2",color="factor(labels)")+geom_point(alpha=0.6)+theme_bw()

In [ ]:
datapc = pd.DataFrame(pca.components_.transpose(),columns=["PC1","PC2"])
datapc['varNames']=newData.columns
ggplot(datapc, aes(x='PC1',y='PC2')) + geom_segment(xend=0, yend=0, arrow=arrow(ends='first'), color='blue')\
+ geom_text(aes(label='varNames'))


# Paso 1: Interpretación K-Means
## Describa las características importantes de cada cluster.
Apoyese en las estadisticas de cada cluster.  Observe que las PCA están basadas principalmente en los scores de usuarios y de la crítica.

# Paso 2 :  DBScan
## Aplique DBSCAN, encuentre el mejor eps para minPts=7 y explique los clusters encontrados

Primero, determinemos el valor de `eps` a utilizar en DBSCAN usando la función de `NearestNeighbors`. Usaremos 7 vecinos para esto.

In [ ]:
from sklearn.neighbors import NearestNeighbors
numNeighboors=7
neighbors = NearestNeighbors(n_neighbors=numNeighboors) #Creando el modelo

# Entrene el modelo con los datos (.fit())


#Use la función kneighbors de neighbors_fit con newData para que retorne las distancis e indices

# OJO: recuerde que .kneighbors() retorna todas las distancias,
# para recuperar las que nos interesan, usar distances[:, numNeighboors-1]

#Ordene las distancias obtenidas usando la función sort del vector

# Grafique las distancias.
# Ejemplo, si sus distancias están en un arreglo llamada `distances`, puede usar
#   pd.DataFrame(distances, index=np.arange(1,len(distances)+1)).plot()



## Paso 3: Ejecute DBScan con el valor de eps seleccionado

In [ ]:
#Creando un objeto de DBSCAN con las condiciones iniciales

from sklearn.cluster import DBSCAN
finalModel = DBSCAN(eps= ... , min_samples=7)
finalModel.fit(....)


In [ ]:
# Revisemos el número de clusters obtenidos, así como el numero de datos en cada cluster
np.unique(finalModel.labels_, return_counts=True)

## Paso 4: Interpretación de DBSCAN
### Interprete los clusters. Describa los clusters obtenidos. ¿Difieren de K-Medias?

Para esto, copie los analisis anteriores de medias por cluster, así como su visualización en las PCA.

# (Recordatorio de comandos)

# DBSCAN
DBSCAN(eps=0.5,min_samples=5, metric='euclidean')<br><br>
Parámetros
* eps: radio de la esfera n-dimensional para la cual se buscan los minPoints.
* minPts: número mínimo de puntos dentro de la región definida por eps para considerar a un punto como crítico (el punto a analizar también se cuenta).
* metric: métrica para el cálculo de la distancia<br> 
scikit-learn => ‘cityblock’, ‘cosine’, ‘euclidean’,‘manhattan’ <br>
scipy.spatial.distance => ‘chebyshev’, ‘correlation’, ‘hamming’, ‘jaccard’, ‘mahalanobis’, ‘minkowski’, ‘seuclidean’,’sqeuclidean’.<br><br>

Atributos
* core_sample_indices_: indices de puntos core.
* components_: Copia de los puntos core.
* labels_: etiquetas de los puntos (-1 implica outlier).<br><br>

Funciones:
* fit(X): entrena el modelo con los parametros asignados.
* fit_predict(X): entrena y devuelve los clusters encontrados con los parametros asignados.

# clase NearestNeighbors
NearestNeighbors(n_neighbors=5, metric='minkowski', p=2)<br>
Parámetros
* n_neighborsint: Números de vecinos a estimar.
* metric: métrica para el cálculo de la distancia.
* p: parámetro de la distancia de minkowski (2 es euclideana). <br><br>

Atributos
* effective_metric_: métrica usada para el cálculo de la distancia.
* effective_metric_params_: Parámetros de la métrica usada para el cálculo de la distancia.
* n_samples_fit_: número de puntos del dataset.<br><br>

Funciones:
* fit(X): Encontrar los vecinos más cercanos de los datos.
* kneighbors(X, n_neighbors): Encontrar los K vecinos más cercano de un punto, retorna tanto los índices como la distancia.
* radius_neighbors(X, radius): Encontrar los vecinos de uno o más puntos que se encuentran dentro de un radio determinado.

# Paso 5: AgglomerativeClustering

Repitamos ahora esto usando cluster jerárquicos.


Dibuje el dendograma de los datos con la función `linkage`.
Pruebe con distintos métodos de calcular la distancia entre cluster

method: ‘ward’, ‘complete’, ‘average’, ‘single’`

In [ ]:
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt


#Entrenando el modelo
modelo = shc.linkage(newData, method= ...)
plt.figure(figsize=(10, 7)) #Seteando el tamaño de la figura

### Hint: cuando graficamos el dendrograma use truncate_mode="lastp" para evitar problemas de memoria.

objeto = shc.dendrogram(modelo, truncate_mode="lastp", p=100) #Generando el dendrograma

Decida un límite en la distancia para cortar el dendograma y generar los clusters.

In [ ]:
clusters=shc.fcluster(modelo,t=...,criterion="distance")



In [ ]:
np.unique(clusters, return_counts=True)

## Paso 6: Interpretación de AglomerativeClustering
### Interprete los clusters. Describa los clusters obtenidos. ¿Difieren de K-Medias? ¿de DBSCAN? ¿Cuál parece mas adecuado?

Para esto, copie los analisis anteriores de medias por cluster, así como su visualización en las PCA.

# AgglomerativeClustering

sklearn tiene varias deficiencias para este modelo. Siendo las más importantes, el definir un número de cluster y no generar un dendrograma en forma sencilla.<br>

Utilizaremos las funciones linkage, dendrogram y dendrogram de la biblioteca scipy.cluster.hierarchy.<br>

La función linkage aprende el modelo definido y retorna un dendrograma<br>
linkage(y, method='single')<br>
Parámetros:
* y: son los datos del modelo
* method: ‘ward’, ‘complete’, ‘average’, ‘single’.<br><br>

Para graficar basta con llamar a la función dendrogram con el modelo aprendido<br>dendrogram(modeloAprendido,p=30,truncate_mode=None)<br>
donde truncate_mode puede ser None (muestra todo el dendrograma) o 'lastp' (corta el dendrograma después de p ramas).

Para extraer los clusters se utiliza la función fcluster
fcluster(Z, t, criterion)<br>
Parámetros:
* Z: el modelo generardo por la función linkage
* t: el valor donde se quiere cortar el dendrograma
* criterion=“distance”. existen otros criterios de corte, pero distance es el que nos permite generar los clusters, basados en la altura que definimos.